**Phân Tích và Xử Lý Dữ Liệu Hiệu Suất Thể Thao: Thông Số Cân Nặng và Kết Quả Tập Luyện**

Kết nối google colab với google drive

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


1. Phân Tích Dữ Liệu Ban Đầu

Đọc dữ liệu từ tệp CSV và in 10 dòng đầu tiên của dataframe.

In [2]:
# Import các thư viện
import pandas as pd
import matplotlib.pyplot as plt

# Đọc dữ liệu từ tệp csv
data = pd.read_csv('/content/drive/MyDrive/Đồ án chuyên ngành khoa học dữ liệu/openpowerlifting.csv')

# In 10 dòng đầu tiên của dataframe với các cột được sắp xếp và không có index khi chưa tiền sử lý
print(data.head(10).to_string(index = False))


 MeetID                   Name Sex  Equipment  Age    Division  BodyweightKg WeightClassKg  Squat4Kg  BestSquatKg  Bench4Kg  BestBenchKg  Deadlift4Kg  BestDeadliftKg  TotalKg Place  Wilks
      0       Angie Belk Terry   F      Wraps 47.0   Mst 45-49         59.60            60       NaN        47.63       NaN        20.41          NaN           70.31   138.35     1 155.05
      0            Dawn Bogart   F Single-ply 42.0   Mst 40-44         58.51            60       NaN       142.88       NaN        95.25          NaN          163.29   401.42     1 456.38
      0            Dawn Bogart   F Single-ply 42.0 Open Senior         58.51            60       NaN       142.88       NaN        95.25          NaN          163.29   401.42     1 456.38
      0            Dawn Bogart   F        Raw 42.0 Open Senior         58.51            60       NaN          NaN       NaN        95.25          NaN             NaN    95.25     1 108.29
      0           Destiny Dula   F        Raw 18.0  Teen 18-

In [3]:
# Import các thư viện
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

Kiểm Tra Giá Trị Thiếu:

Kiểm tra số lượng giá trị thiếu trong mỗi cột.

Sử dụng phương pháp forward fill để thay thế giá trị thiếu và xử lý các giá trị NaN khác.

In [4]:
# Kiểm tra các giá trị thiếu
print("Số lượng giá trị thiếu trong mỗi cột:")
print(data.isna().sum())

# Sử dụng phương pháp forward fill để thay thế các giá trị thiếu
data = data.ffill()

Số lượng giá trị thiếu trong mỗi cột:
MeetID                 0
Name                   0
Sex                    0
Equipment              0
Age               239267
Division           15843
BodyweightKg        2402
WeightClassKg       3812
Squat4Kg          385171
BestSquatKg        88343
Bench4Kg          384452
BestBenchKg        30050
Deadlift4Kg       383614
BestDeadliftKg     68567
TotalKg            23177
Place               1092
Wilks              24220
dtype: int64


Xử Lý Dữ Liệu Nhiễu và Bất Thường:

Loại bỏ các hàng trùng lặp và giá trị âm trong cột 'Squat4Kg'.

Thay thế giá trị NaN trong các cột số bằng giá trị trung bình của từng cột.

In [5]:
# Loại bỏ các hàng trùng lặp
data = data.drop_duplicates()

# Thay thế giá trị NaN trong cột cụ thể bằng giá trị trung bình
data['Squat4Kg'].fillna(data['Squat4Kg'].mean(), inplace=True)

# Loại bỏ các hàng có bất kỳ giá trị NaN nào
data_cleaned = data.dropna()

# Thay thế giá trị NaN trong tất cả các cột số bằng giá trị trung bình của từng cột
numeric_columns = ['Squat4Kg', 'Bench4Kg', 'Deadlift4Kg']
data[numeric_columns] = data[numeric_columns].fillna(data[numeric_columns].mean())

# Kiểm tra các giá trị âm trong cột 'Squat4Kg'
negative_squat_values = data[data['Squat4Kg'] < 0]
print("Các giá trị âm trong Squat4Kg:")
print(negative_squat_values)

# Loại bỏ giá trị âm trong cột 'Squat4Kg'
data = data[data['Squat4Kg'] >= 0]

<ipython-input-5-394bf9efec41>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Squat4Kg'].fillna(data['Squat4Kg'].mean(), inplace=True)


Các giá trị âm trong Squat4Kg:
        MeetID                    Name Sex  Equipment   Age     Division  \
5            0         Courtney Norris   F      Wraps  28.0  Open Senior   
6            0           Maureen Clary   F        Raw  60.0    Mst 60-64   
7            0           Maureen Clary   F        Raw  60.0  Open Senior   
8            0  Priscilla Sweat Pardue   F      Wraps  52.0        50-54   
9            0  Priscilla Sweat Pardue   F        Raw  52.0       Senior   
...        ...                     ...  ..        ...   ...          ...   
386024    8477              Rheta West   F  Multi-ply  33.0         Open   
386025    8477             Beth Thomas   F  Multi-ply  33.0         Open   
386026    8477        Nina Kondrasheva   F  Multi-ply  33.0         Open   
386027    8477              Tara Green   F  Multi-ply  33.0         Open   
386028    8477        Allison Lockhart   F  Multi-ply  33.0         Open   

        BodyweightKg WeightClassKg  Squat4Kg  BestSquatK

Chuẩn Hóa Dữ Liệu:

Chuẩn hóa cột 'Age' nếu không có giá trị NaN.

Xử lý lỗi định dạng trong cột 'Sex' (chuyển thành chữ hoa và loại bỏ khoảng trắng).

In [6]:
# Chuẩn hóa dữ liệu chuẩn hóa cột 'Age'
if data['Age'].isnull().sum() == 0:
    data['Age'] = (data['Age'] - data['Age'].mean()) / data['Age'].std()
else:
    print("Cột 'Age' chứa giá trị NaN. Hãy xử lý các giá trị NaN trước khi chuẩn hóa.")

# Xử lý lỗi dữ liệu như lỗi định dạng
data['Sex'] = data['Sex'].str.strip().str.upper()

# In 10 dòng đầu tiên của DataFrame sau khi đã tiền sử lý xong
print("Dữ liệu đã xử lý (10 dòng đầu tiên):")
print(data.head(10).to_string(index=False))

Dữ liệu đã xử lý (10 dòng đầu tiên):
 MeetID             Name Sex  Equipment       Age    Division  BodyweightKg WeightClassKg   Squat4Kg  BestSquatKg    Bench4Kg  BestBenchKg  Deadlift4Kg  BestDeadliftKg  TotalKg Place  Wilks
      0 Angie Belk Terry   F      Wraps  1.016017   Mst 45-49         59.60            60  10.309721        47.63   93.708581        20.41   118.643786           70.31   138.35     1 155.05
      0      Dawn Bogart   F Single-ply  0.687373   Mst 40-44         58.51            60  10.309721       142.88   93.708581        95.25   118.643786          163.29   401.42     1 456.38
      0      Dawn Bogart   F Single-ply  0.687373 Open Senior         58.51            60  10.309721       142.88   93.708581        95.25   118.643786          163.29   401.42     1 456.38
      0      Dawn Bogart   F        Raw  0.687373 Open Senior         58.51            60  10.309721       142.88   93.708581        95.25   118.643786          163.29    95.25     1 108.29
      0     D

Loại Bỏ Các Cột Không Cần Thiết:

Xóa các cột không cần thiết như 'MeetID', 'Sex', và 'Equipment'.

In [7]:
# Xóa các cột không cần thiết
# Xóa cột 'MeetID'
if 'MeetID' in data.columns:
    data = data.drop(columns=['MeetID'])

# Xóa cột 'Sex'
if 'Sex' in data.columns:
    data = data.drop(columns=['Sex'])

# Xóa cột 'Equipment' nếu đã chuyển đổi thành các biến số nhị phân
if 'Equipment' in data.columns:
    data = data.drop(columns=['Equipment'])


Kiểm Tra Mối Quan Hệ Giữa Các Đặc Trưng:

In ma trận tương quan giữa các đặc trưng để kiểm tra sự liên kết.

In [8]:
# Kiểm tra mối quan hệ giữa các đặc trưng
print("Ma trận tương quan:")
correlation_matrix = data[['Age', 'BodyweightKg', 'Squat4Kg', 'Bench4Kg', 'Deadlift4Kg', 'TotalKg']].corr()
print(correlation_matrix)

Ma trận tương quan:
                   Age  BodyweightKg  Squat4Kg  Bench4Kg  Deadlift4Kg  \
Age           1.000000      0.039123 -0.244473 -0.124789     0.114558   
BodyweightKg  0.039123      1.000000  0.061355 -0.001121    -0.005609   
Squat4Kg     -0.244473      0.061355  1.000000  0.059429    -0.108082   
Bench4Kg     -0.124789     -0.001121  0.059429  1.000000     0.313023   
Deadlift4Kg   0.114558     -0.005609 -0.108082  0.313023     1.000000   
TotalKg      -0.117118      0.396861  0.029956  0.049474     0.032282   

               TotalKg  
Age          -0.117118  
BodyweightKg  0.396861  
Squat4Kg      0.029956  
Bench4Kg      0.049474  
Deadlift4Kg   0.032282  
TotalKg       1.000000  


2. Kiểm Tra Tiền Xử Lý Dữ Liệu

Chia Dữ Liệu Thành Tập Huấn Luyện và Tập Kiểm Tra:

Tách dữ liệu thành tập huấn luyện và tập kiểm tra.

Chuẩn hóa dữ liệu bằng cách sử dụng StandardScaler.

In [10]:
# Chia dữ liệu thành tập huấn luyện và tập kiểm tra
X = data[['Age', 'BodyweightKg', 'Squat4Kg', 'Bench4Kg', 'Deadlift4Kg']]
y = data['TotalKg']  # có thể chọn cột mục tiêu khác
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Chuẩn hóa dữ liệu
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# In số lượng mẫu trong tập huấn luyện và tập kiểm tra
print(f"Số lượng mẫu trong tập huấn luyện: {X_train.shape[0]}")
print(f"Số lượng mẫu trong tập kiểm tra: {X_test.shape[0]}")

Số lượng mẫu trong tập huấn luyện: 187872
Số lượng mẫu trong tập kiểm tra: 46968


Phân Tích Nhân Tố (Factor Analysis):

Áp dụng phân tích nhân tố để xác định số lượng yếu tố và hiển thị ma trận tải trọng.

In [11]:
from sklearn.decomposition import FactorAnalysis

# Xác định số lượng yếu tố (components)
fa = FactorAnalysis(n_components=2)
X = data[['Age', 'BodyweightKg', 'Squat4Kg', 'Bench4Kg', 'Deadlift4Kg']]
X_factors = fa.fit_transform(X)

# Hiển thị ma trận tải trọng (loading matrix)
print("Ma trận tải trọng (loading matrix):")
print(pd.DataFrame(fa.components_, columns=X.columns, index=['Factor1', 'Factor2']))


Ma trận tải trọng (loading matrix):
              Age  BodyweightKg   Squat4Kg   Bench4Kg  Deadlift4Kg
Factor1 -0.118303      0.135664   7.687205 -26.840622  -105.202829
Factor2  0.646718      0.493235 -24.794492 -21.404247    -0.297526


Mô Hình Hóa Hỗn Hợp (Gaussian Mixture Modeling):

Tạo mô hình Gaussian Mixture để phân nhóm dữ liệu.

Dự đoán các cụm (clusters) và thêm cột cụm vào dữ liệu.

Hiển thị số lượng điểm trong mỗi cụm.

In [12]:
from sklearn.mixture import GaussianMixture

# Tạo mô hình Gaussian Mixture
gmm = GaussianMixture(n_components=3, random_state=42)
gmm.fit(X)

# Dự đoán các cụm (clusters)
clusters = gmm.predict(X)

# Thêm cột cụm vào dữ liệu
data['Cluster'] = clusters

# Hiển thị số lượng điểm trong mỗi cụm
print("Số lượng điểm trong mỗi cụm:")
print(data['Cluster'].value_counts())


Số lượng điểm trong mỗi cụm:
Cluster
1    164814
0     60633
2      9393
Name: count, dtype: int64


3. Thống Kê Mô Tả


In thống kê mô tả cho các cột số để đánh giá các thuộc tính của dữ liệu sau khi xử lý.

In [14]:
# Thống kê mô tả cho các cột số
print("Thống kê mô tả của dữ liệu:")
print(data.describe())

Thống kê mô tả của dữ liệu:
                Age   BodyweightKg       Squat4Kg    BestSquatKg  \
count  2.348400e+05  234840.000000  234840.000000  234840.000000   
mean   8.865143e-17      88.314073     178.485670     185.587250   
std    1.000000e+00      23.894458      69.347729      87.309763   
min   -1.744593e+00      15.880000      10.309721    -477.500000   
25%   -6.929319e-01      71.100000     123.000000     136.080000   
50%   -4.300166e-01      86.180000     202.500000     183.700000   
75%    8.845598e-01     103.000000     202.500000     232.500000   
max    4.171001e+00     230.800000     450.000000     573.790000   

            Bench4Kg    BestBenchKg    Deadlift4Kg  BestDeadliftKg  \
count  234840.000000  234840.000000  234840.000000   234840.000000   
mean      123.012393     120.622804     158.830206      199.045797   
std        84.537894      63.470705     106.950910       65.270609   
min      -360.000000    -522.500000    -461.000000     -410.000000   
25%      